<a href="https://colab.research.google.com/github/SandyDRawat/Self_Projects/blob/main/Novel%20Automation%20Tool/With%20Translation/novel_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ebooklib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=261f0cef80fd887b3fbe75e2f0693f000ba48cfcfa0ffd0086ca5642ac54f359
  Stored in directory: /root/.cache/pip/wheels/0f/38/cc/a3728bb72a315d9d8766fb71d362136372066fc25ad838f8fa
Successfully built ebooklib


In [ ]:
import requests
from bs4 import BeautifulSoup
from ebooklib import epub
from google_trans_new import google_translator

In [ ]:
def translate_text(text):
       translator = google_translator(timeout=5)
       if type(text) is not str:
           translate_text = ''
           for substr in text:
               translate_substr = translator.translate(substr,'hi')
               translate_text += translate_substr
       else:
           translate_text = translator.translate(text,lang_tgt='hi', lang_src='en')
       return translate_text


In [ ]:
def scrapper(url):
    req = requests.get(url)
    if req.status_code == 200:
        req.encoding = 'utf-8'
        soup = BeautifulSoup(req.text, "html.parser")
        next_chapter_link = soup.find('a', id='next_chap')['href']
        paragraphs = soup.find_all('p')
        chapter_content = ''
        for p in paragraphs:
            #a = translate_text(p)
            chapter_content += str(p)  # Convert to string to preserve HTML tags

        return chapter_content, next_chapter_link
    else:
        print("Failed to retrieve content from the URL.")
        return None, None

# Function to create an ePub file
# Function to create an ePub file
def create_epub(chapter_contents, novel_name, start_chapter, num_chapters):
    end_chapter = start_chapter + num_chapters - 1
    epub_name = f"{novel_name}_{start_chapter}_to_{end_chapter}.epub"
    book = epub.EpubBook()
    book.set_identifier(epub_name)
    book.set_title(novel_name)
    book.add_author("Author Name")  # Replace with actual author name
    for i, content in enumerate(chapter_contents, start=start_chapter):
        chapter_number =chapter_numbers[i]  # Adjusting chapter number
        chapter_content_with_number = f'<h1>Chapter {chapter_number}</h1>\n{content}'
        chapter = epub.EpubHtml(title=f'Chapter {chapter_number}', file_name=f'chapter_{chapter_number}.xhtml', lang='en')
        chapter.content = chapter_content_with_number
        book.add_item(chapter)
        book.toc.append(chapter)
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())
    style = 'body { font-family: Times, Times New Roman, serif; }'
    nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)
    book.add_item(nav_css)
    book.spine = ['nav'] + book.toc
    epub.write_epub(epub_name, book)

In [ ]:
def translate_chap(chap_contents):
    content = ''
    #print(type(chap_contents))
    #print(chap_contents)
    contents = chap_contents.split('</p>')
    #print(chap_contents[3])
    a = ''
    for cont in contents:
        #print(cont)
        a += cont
        if len(a) >4000:
            content += translate_text(a)
            a = ''
    content += translate_text(a)
    return content

In [ ]:
url = input("URL of The Starting Chapter: ")
num_chapters_to_scrape = int(input("Number of Chapters: "))

# Extract novel name from URL
novel_name = url.split('novel-book/')[1].split('/chapter')[0]

# Initialize lists to store chapter content and chapter numbers
chapter_contents = []
chapter_numbers = []

# Start scraping loop
for i in range(num_chapters_to_scrape):
    url0 = url
    chapter_content, url = scrapper(url)
    if chapter_content:
       # print(chapter_content)
       # print(translated_chap_content)
        chapter_contents.append(translate_chap(chapter_content))

        chapter_numbers.append(int(url0.split('/')[-1].split('-')[1]))
    url = url.rsplit('-', 1)[0] + f'-{i + 2}'  # Correcting the URL for the next chapter
   # print(i)

# Get start chapter and end chapter
start_chapter = min(chapter_numbers)
end_chapter = start_chapter + num_chapters_to_scrape - 1

#print(chapter_contents)
# Export the content to ePub
create_epub(chapter_contents, novel_name, start_chapter, num_chapters_to_scrape)


URL of The Starting Chapter: https://novelbin.englishnovel.net/novel-book/dimensional-descent/chapter-1
Number of Chapters: 1
